## ATTPP (Actor-based Trait/Token Programming Paradigm)
I've come up with this pattern for PL. The idea is to have a set() of flags - called traits that define the properties and methods of a class. Instead of defining "classes" of traits or subclassing explicitely, the traits are grouped into so-called "stereotypes", simple set()s of traits that can interact with each other.

Traits basically implement the flyweight and visitor pattern while simplifying the handling on the user side, overcoming all the problems of multiple inheritance.

There are some issues here:

### Traits as enums?
The first approach used a Trait(Enum) with name->Implementation, which seemed to work at first but being an enum, this mapping could never be changed, but worse: it never can be extended while all traits using the same implementation map to the same name.

### Traits as classes?
Classes in module-namespace act like enums (they are singletons), but unlike enums, their implementation can be changed on the fly and classes can be added.
This may be a feasible solution, but it adds the temptation to have state in the Trait() which actually belongs to the Thing() and should be shared with other traits, adding unnecessary complexity.

### Traits as static methods?
If we accept that traits should have no state by itself, we could take the functional route and implement them as externalized methods of Thing. This can be done by having a Trait class which just acts as a namespace for @staticmethods, taking Thing() as parameter that can be introspected. The Thing.bag is a very powerful datastructure that can be used to establish locks and loops across traits.

### Actors!
Almost forgot the purpose of the exercise: to separate the implementation from the Thing to easily switch it on the fly, depending on the context. So, all we need to do is to go with the first approach - traits as enums - but then have an Actor/Visitor class that implements the details, simply checking the thing for traits and tokens. It's really simple and elegant even. As traits and tokens are enums, their namespaces are seperated, typos are caught instantly and the resulting code is clearly readable with a minimum of noise. 

If there are things that should be stored with a thing but can't easily be coded as a trait or token (for instance the coordinates of a portal), it's best to have an external WeakKeyDictionary store the data associated with the thing (flyweight pattern) so that traits and tokens can be used as "metadata".

## FSM!
Implementing finite state machines with enums really is cool and simple, as shown elsewhere. To implement complex kinds of FSM you need some kinds of tokens - but wait! Traits are enums and they work in tandem with tokens! This smells like a delicious cake of pattern-goodness.

## And there we are.. reinventing the wheel
https://en.wikipedia.org/wiki/Automata-based_programming


## But my wheels are better than yours!
So, I've reinvented the wheels (like so many others), but here is why this approach is so much better than most of the others. The StateMachine itself implements only state and prevents invalid transitions, implementing basic functionality but other than that it's locked - all the rest of the logic is built outside in an actor class or module-level functions, which, ideally, only flips the flux_to switch and nothing else.

This allows for the most simple approach for representing state: Enums, sets in a \__slots\__-locked class. This means that set-operations can be used for logic and with enums, typos never translate into logic errors and also means less typing while being consistent across modules.

Transitions only reply with True or False to remove, but logging and warnings are also done in order to trace back problems if there are any.

In [ ]:
from flux import StateMachine
from enum import Enum

K = Enum("Kettle", "empty full boiling")

kettle = StateMachine(name="kettle",
                      states={K.empty: {K.full},
                              K.full: {K.boiling},
                              K.boiling: {K.empty}},
                      initial=K.empty)

class Actor:
    def fill(self, kettle):
        kettle.flux_to(K.full)
    
    def boil(self, kettle):
        kettle.flux_to(K.boiling)
        
    def pour(self, kettle):
        kettle.flux_to(K.empty)
        
cook = Actor()

cook.fill(kettle)
cook.boil(kettle)
cook.pour(kettle)
cook.boil(kettle)

Ok, I admit actor-trait based programming might seem a bit counter-intuitive, so I've added a way to call functions on transition, which works pretty much like regular callbacks in GUIs and networking like twisted.

Imagine you have a switch that can be ON or OFF or screwed up. To model this, first you do

In [ ]:
from enum import Enum
from flux import StateMachine
IS = Enum("IS", "on off dead")

to define the possible states aka "alphabet".
Then you define the possible state transitions:

In [ ]:
switch = StateMachine(states={IS.off: {IS.on, IS.off, IS.dead},
                                 IS.on: {IS.off, IS.on, IS.dead},
                                 },
                          initial=IS.off)

it simply can't get easier than that. 
Notice that IS.dead doesn't have any transitions defined, which makes it a final state.
Now, to switch the switch ON, you just do

    switch.flux_to(IS.on)
    switch.flux_to(IS.off)

If you want to have some action attached to this, you can define it either in general for exiting the previous state (IS.off), in general for entering the next state (IS.on) OR you can define an action for the transit between specific states, between exit and enter. 
You can define all three even - then the order of executation is 1) exit 2) transit 3) enter. The action can be any decorated function or inline lambda.

In [ ]:
switch.reg_exit(IS.off, lambda e: print("not OFF anymore!"))

@switch.reg_enter(IS.on)
def _(e):
    print("it's getting brighter..")
    print("bring it ON!")

switch.reg_transition(IS.off, IS.on, 
                      lambda e: print("not sure yet"))

Notice that only one callback function can be defined for any one of the three steps. This is a deliberate design decision. If any code could register a callback function for a state transition anywhere, the order of execution would become very hard to predict - especially if third parties are involved. The price for arbitrary callback chaining would be naming each and every function - so no lambdas and throw-away functions - and a complex maintenance toolchain for debugging and keeping track of what is calling what with what parameters. Too much unnecessary hassle. 
If you really want to chain functions on transition, have a single function that branches out. This way you have all complexity in one place and less confusion overall.
In case a third party really does register a callback, this way you can be certain that there only can be one such function and if you want to override it, you know exactly what to search for, and call from your own code.

In [ ]:
switch.flux_to(IS.on)

## pluggable adapter, delegator or trait with flyweight
this is an exceptionally useful combination of patterns for several reasons:
firstly, no subclassing is necessary. this means there are no collisions of methods and attributes, reducing complexity a lot. It also means Entities and delegators can easily be subclassed for other reaons as long as the method signatures are compatible (kwargs, not positional for cooperative super()!). 
use NotImplemented instead of NotImplementedError for quick check without exceptions. the delegation class should be just a set of related methods with no internal state. this should enable easy functional testing of the delegation with no side effects. last but not least, handling the delegation as a property allows quick and uncomplicated hotplugging of delegators with no change of call-semantics whatsoever. this means the calling side can't and shouldn't care about the current implementation (it could change at any time!), enforcing loose coupling in calling. this also helps in keeping the code at the calling side real simple since only the Entity with a property is called. last but not least, extra needed state associated with the delegator can easily be stored outside in a WeakKeyDict or WeakValueDict so that no unnecessary housekeeping, overcrowding of the Entity namespace or internal state inside the delegator is needed, enabling the flyweight pattern (very little internal state, no \__dict\__, and \__slots\__ instead).

## SSCCE for trait-token paradigm programming 
(Short, Self Contained, Correct (Compilable), Example)
This is the simplest example I could come up with to illustrate the trait-oriented programming pattern I developed.

1. First, an enum is declared as a contract between all parties involved. All parties promise to use *only* the enum as traits **nothing else**. 
2. The Thing class only has traits (as a set of enum) to define behaviour and tokens (as a counter of enum) for a minimum of state. If more state is needed, it can/should be externalized via properties operating on global dicts.
3. Any behaviour is externalized into Actor classes or simple handler functions with no state. They are envoked from an accept function in the Thing class, which passes the instance to the Actor/handler. The Actor/handler introspects the traits and tokens of the thing and operates accordingly.

This clearly separates state from functionality: Thing represents state (traits, counter and interface to global dicts), Actor implements functionality without state.
The first and obvious advantage of this pattern is that it is trivial to expand and recombine functionality on the fly without hardcoding classes and subclasses. Since traits are realized as sets of enums, handling of "classes" is simple, safe and fast. This extraordinary combination of qualities is hard to achieve otherwise.

Another obvious advantage of this pattern is that it implements the foundation of finite state machines (FSM), making it trivial to realize any type of any complexity of state machine from here.

## Linchpin: case handling
The pattern stands or falls depending on how cases are handled in the Actor class, realizing the traits. If there are many cases (more than 2), it is advisable to use the dictionary-switch-pattern. Depending on how the traits actually should be handled, it's possible to use tuples of bool as keys to identify cases and test against traits once (that's one of the most efficient ways). This is recommended for traits that interact. If there are traits that are independent of each other, they can be handled in a for-loop over the dict of cases, this time with frozensets of traits to test against.
Since traits are implemented as sets, many complex tests (for instance supersets) can be realized efficiently without making the code complicated.

In [4]:
from enum import Enum
from flux import TraitStateMachine, can

IS = Enum("State", "closed open init final")
T = Enum("Token", "opened hitpoint")
DO = Enum("Action", "open close hit write enter")

door = TraitStateMachine(states={IS.open: {IS.closed, IS.final},
                            IS.closed: {IS.open, IS.final},
                            IS.init: {IS.open}},
                    initial=IS.init,
                    stateful_traits={IS.open: {DO.close, DO.hit, DO.write, DO.enter},
                                   IS.closed: {DO.open, DO.hit, DO.write},
                                    IS.init: {DO.open}}, 
                   bag={T.hitpoint: 3}
                    )

class Guy:
    @can(DO.close)
    def close(self, thing):
        thing.flux_to(IS.closed)
        print("guy closes the door")
    
    @can(DO.enter)
    def enter(self, thing):
        print("guy walks through the door")
    
    @can(DO.write)
    def write(self, thing):
        print("guy writes on the door")
    
    @can(DO.open)
    def open(self, thing):
        thing.flux_to(IS.open)
        thing.bag[T.opened] += 1
        print("guy opens the door")
    
    @can(DO.hit)
    def fubar(self, thing):
        thing.bag[T.hitpoint] -= 1
        print("he slams against the door")
        
        if thing.bag[T.hitpoint] <= 0:
            thing.flux_to(IS.final)
            print("guy broke the door") 

@can(DO.close)
def close(thing):
    thing.flux_to(IS.closed)


guy = Guy()
guy.open(door)
close(door)
guy.fubar(door)

de5cd8f4-04a0-4368-9b38-31655e5c014c now State.open
de5cd8f4-04a0-4368-9b38-31655e5c014c now State.closed


guy opens the door
he slams against the door


In [6]:
from flux import TraitMachine, Collector
from enum import Enum

E = Enum("Eign", "a b c")
S = Enum("State", "x y")

m = TraitMachine(initial=S.x, default=S.x,
                states={S.x: {S.y},
                        S.y: {S.x}},
               trait_states={frozenset({E.a, E.b, E.c}): S.y},
               )

m.flux_to(S.y)
m()
m.eigns = {E.a, E.b, E.c}
m()

c = Collector(initial=S.x, default=S.x,
               states={S.x: {S.y},
                       S.y: {S.x}},
             )
c[(m, S.y)] = S.y

c.conditions

d1427a13-74bf-443f-99c3-c66f0222769e now State.y
d1427a13-74bf-443f-99c3-c66f0222769e now State.x
/media/sf_Dropbox/jupyter/flux/flux.py:159: UserWarning: no valid transition from State.x to State.x
  warn(f"no valid transition from {self.state} to {to_state}", UserWarning)


OrderedDict([(frozenset({(d1427a13-74bf-443f-99c3-c66f0222769e in State.x with {<State.x: 1>: {<State.y: 2>}, <State.y: 2>: {<State.x: 1>}},
                          <State.y: 2>)}),
              <State.y: 2>)])

In [ ]:
from enum import Enum
from flux import StateMachine

IS = Enum("IS", "open closed broken")
DO = Enum("DO", "open close destroy")


class Door(StateMachine):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.closed = 0
        
    def accept(self, aim, *args):
        def close():
            self.flux_to(IS.closed)
            self.closed += 1
            
        cases = {self.state is IS.closed and aim is DO.open: lambda: self.flux_to(IS.open),
                 self.state is IS.open and aim is DO.close: lambda: close(),
                 aim is DO.destroy: lambda: self.flux_to(IS.broken)}
        try:
            cases[True]()
            return True
        except KeyError:
            return False
        
door = Door(states={IS.open: {IS.closed, IS.broken},
                    IS.closed: {IS.open, IS.broken}},
                   initial=IS.closed,
                   name="door")
class Actor:
    def open(self, thing):
        thing.accept(DO.open)
        
    def close(self, thing):
        thing.accept(DO.close)
    
    def destroy(self, thing):
        thing.accept(DO.destroy)
    
    def mad(self, thing):
        for x in range(3):
            thing.accept(DO.open, x)
            thing.accept(DO.close, x)
    
guy = Actor()

guy.open(door)
guy.mad(door)
guy.destroy(door)
guy.open(door)

door.state
door.closed

Since we figured out how to dispatch on different sets of traits, let's see how easy it is to implement the stateful GUI of PL.
Okay, it seems there's an even more elegant way than using dispatch here. If the states are implemented as classes - not enums - it's possible to simply use @staticmethod for the states and have all problems fixed at the same time: 1) The state definition is a well-named singleton with auto-completion 2) it's easy to "dispatch" methods per state - states don't need to be instanciated but can be decorated with @staticmethod so they can be called just like that - exactly the way the dispatch would work.

This is an even more elegant solution since it takes care of dispatching state-dependent methods with no boilerplate at all. The state transition definition is clean and easy as it should be while it's still trivial to have state-dependent traits as flags and the state manager also has a bag to keep track of any additional info.

In [ ]:
from flux import StateMachine

class State:
    @staticmethod
    def display_msg(msg):
        print(msg)
    
class Loading(State):
    pass

class Login(State):
    @staticmethod
    def display_msg(msg):
        print("login", msg)

class Signup(State):
    pass

class Playing(State):
    pass

class Exit(State):
    pass

gs = StateMachine(states={Loading: {Login},
                          Login: {Signup, Playing, Exit},
                          Signup: {Login},
                          Playing: {Exit}},
                 initial=Loading)

@gs.reg_enter(Signup)
def _():
    render.hide()
    base.setBackgroundColor(1, 1, 1)
    base.setFrameRateMeter(False)
    
gs.reg_enter(Exit, lambda: base.careen())

gs.state.display_msg("hello")
gs.flux_to(Login)
gs.state.display_msg("hello")


In [ ]:
from flux import StateMachine
from enum import Enum
#from bridge import render, base

GS = Enum("GUIState", "loading login signup playing exit")
    
gs = StateMachine(states={GS.loading: {GS.login},
                          GS.login: {GS.signup, GS.playing, GS.exit},
                          GS.signup: {GS.login},
                          GS.playing: {GS.exit}},
                 initial=GS.loading)

@gs.reg_enter(GS.loading)
def _(e):
    render.hide()
    base.setBackgroundColor(1, 1, 1)
    base.setFrameRateMeter(False)
    
gs.reg_enter(GS.exit, lambda e: base.careen())

In [ ]:
from enum import Enum
from flux import StateMachine

S = Enum("State", "incomplete invalid wont_fix in_progress new fix_committed fix_released")

BugStatus = StateMachine(states={S.new: {S.incomplete, S.invalid, S.wont_fix, S.in_progress},
                          S.incomplete: {S.new, S.wont_fix},
                          S.invalid: {S.new},
                          S.wont_fix: {S.new},
                          S.in_progress: {S.new, S.fix_committed},
                          S.fix_committed: {S.in_progress, S.fix_released},
                          S.fix_committed: {S.new}},
                  initial=S.new)

In [ ]:
from flux import StateMachine, Collector
from enum import Enum
S = Enum("State", "on off dead")

a = StateMachine(states={S.on:{S.on, S.off}, 
                         S.off:{S.off, S.on}},
                initial=S.on,
                name="a")

b = a.clone(name="b")
c = a.clone(name="c")

x = Collector(states={S.on: {S.dead, S.on}},
              initial=S.on,
             default=S.dead,
             name="x")

y = x.clone(name="y")

y[(a, S.on), (b, S.on)] = S.on
x[(a, S.on), (b, S.on), (c, S.on), (y, S.on)] = S.on

x()

In [ ]:
from flux import StateMachine, Collector
from enum import Enum
S = Enum("State", "on off dead")

a = StateMachine(states={S.on:{S.on, S.off}, 
                         S.off:{S.off, S.on}},
                initial=S.on,
                name="a")

x = Collector(states={},
             initial=S.dead,
              name="x")

y = x.clone(name="y")
x[y, S.on] = S.on
y[x, S.on] = S.on

Networking benefits greatly from using statemachines. For instance, both parties of a connection can have a reliable representation of each other's state, which simplifies communication a lot.

In [ ]:
from flux import StateMachine, Collector
from enum import Enum

ES = Enum("ESState", "starting greeting login playing exitting disconnected")

ESClient = StateMachine(initial=ES.starting,
                       states={ES.starting:{ES.greeting},
                              ES.greeting:{ES.login},
                              ES.playing:{ES.exitting, ES.disconnected},
                              ES.disconnected: {ES.playing, ES.exitting}})



## example from http://zguide.zeromq.org/page:all#Serializing-Your-Data

~~~~
nom-protocol    = open-peering *use-peering

open-peering    = C:OHAI ( S:OHAI-OK / S:WTF )

use-peering     = C:ICANHAZ
                / S:CHEEZBURGER
                / C:HUGZ S:HUGZ-OK
                / S:HUGZ C:HUGZ-OK
~~~~

In [ ]:
from flux import StateMachine
from enum import Enum

C = Enum("Client", "OHAI ICANHAZ HUGZ CHEEZBURGER HUGZ_OK BAI")
S = Enum("Server", "OHAI_OK WTF HUGZ_OK HUGZ BAI")

nom = StateMachine(states={C.OHAI: {S.OHAI_OK, S.WTF},
                            S.OHAI_OK: {C.ICANHAZ, C.HUGZ},
                            C.HUGZ: {S.HUGZ_OK, S.BAI},
                            S.HUGZ: {C.HUGZ_OK},
                            S.HUGZ_OK: {C.ICANHAZ, C.HUGZ},
                            S.BAI: {C.BAI},},
                  initial=C.OHAI)
nom.flux_to(S.OHAI_OK)
nom.flux_to(C.HUGZ)
nom.flux_to(S.HUGZ_OK)
nom.next_possible_states

As you can see, the protocol defined above is basically 1:1 translated into working code. Furthermore, issues with the specification were easy to identify and fix by playing around with the statemachine and discovering the transitions step by step.

While StateMachines are really interesting in many situations, a simple TraitBag often can just do the trick.

In [1]:
from flux import TraitBag, can
from enum import Enum

T = Enum("Trait", "a b c")

thing = TraitBag(in_traits=[T.a, T.c])

@can(T.a)
def do_it(x):
    print("that did it")
    
do_it(thing)

thing.traits.discard(T.a)

do_it(thing)

that did it


/media/sf_Dropbox/jupyter/flux/flux.py:483: UserWarning: do_it can't be called, a6b3f has no Trait.a
  warn(f"{func.__qualname__} can't be called, {thing} has no {in_trait}")


And of course, you can implement Fluxx the card game with fluxx.

### Game Play
It all begins with one basic rule: Draw one card, Play one card. You start with a hand of three cards... add the card you drew to your hand, and then choose one card to play, following the directions written on your chosen card. As cards are drawn and played from the deck, the rules of the game change from how many cards are drawn, played or even how many cards you can hold at the end of your turn.

In [ ]:
from fluxx import TraitStateMachine
from enum import Enum

C = Enum("Card", "win lose nothing")
G = Enum("Phase", "draw play check end")
P = Enum("Player", "A B C")

game = TraitStateMachine(states={G.draw: {G.play},
                                G.play: {G.check},
                                G.check: {G.draw}},
                        initial=G.draw)
